In [544]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [545]:
# Download the latest SO dataset here: https://drive.google.com/open?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV
data_2019 = pd.read_csv("developer_survey_2019/survey_results_public.csv", low_memory=False)

In [546]:
to_drop = ["Respondent", "OpenSource", "CareerSat", "JobSat", "JobSeek", "ResumeUpdate", "SurveyLength", "SurveyEase", "WelcomeChange", "EntTeams", "ScreenName", "LastIn", "SO", "Blockchain", "WorkChallenge", "BetterLife", "OffOn", "Currency", "CompTotal", "CompFreq", "MainBranch", "ITPerson"]

In [547]:
def col_drop(df, to_drop):

    df_dropped = df.copy()

    for flag in to_drop:
        try:
            df_dropped.drop([x for x in df_dropped.columns if flag in x], axis=1, inplace=True)
        except:
            pass

    return df_dropped

In [548]:
def string_replace(s):
    try:
        s = float(s)
    except:
        s = -1000

    return s

In [549]:
def text_clean(text):
    text = str(text).replace(" ", "_").replace("-", "_").replace(
        ",", "_").replace(".", "").replace("+", "p").replace("#", "s").replace(
            "/", "_").replace("'", "").replace("ʼ", "").replace(
                "(", "_").replace(")", "_").replace("’", "").replace(
                    "__", "_").replace("__", "_").replace("“", "").replace(
                        "”", "").replace(":", "_").replace("&", "_").lower()

    return text

In [559]:
def create_controls(df, exclude):

    controls = {}

    for col in df.columns:
        if col != exclude:
            controls[col] = {"omitted": text_clean(pd.Series([x for sub in list(data_2019[col].apply(text_clean).apply(lambda x: str(x).split(";"))) for x in sub]).value_counts().idxmax()), "controls": list(set([x for sub in list(data_2019[col].apply(text_clean).apply(lambda x: str(x).split(";"))) for x in sub]))}
    return controls

In [563]:
def design_matrix(df, controls):
    dm = df.copy()
    
    for control in controls.keys():
        dm[control] = dm[control].apply(text_clean)

        if control in num_columns:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = (dm[control] == c) * 1

        else:
            for c in controls[control]["controls"]:
                dm[control+"_"+c] = dm[control].apply(lambda x: c in str(x).split(";")) * 1

        dm.drop(control, axis=1, inplace=True)
        dm.drop(control+"_"+controls[control]["omitted"], axis=1, inplace=True)
    
    return dm

In [552]:
data_2019 = col_drop(data_2019, to_drop)
print("Success!")
print(data_2019.head().T.shape)

Success!
(51, 5)


In [553]:
# Only consider those with income between $10,000 and $250,000
data_2019 = data_2019[(data_2019["ConvertedComp"] >= 10000) & (data_2019["ConvertedComp"] <= 250000)]
data_2019["ConvertedComp"] = np.log(data_2019["ConvertedComp"])
data_2019 = data_2019.rename(columns = {"ConvertedComp": "Income"})

# Only consider US respondents
data_2019 = data_2019[data_2019["Country"] == "United States"]

# Only consider 18+ respondents
data_2019 = data_2019[data_2019["Age"] >= 18]

# Only consider respondents in the workforce
data_2019 = data_2019[data_2019["Employment"] != "Retired"]
data_2019 = data_2019[data_2019["Employment"] != "Not employed, and not looking for work"]

data_2019 = data_2019[data_2019["WorkWeekHrs"] >= 5]

# Only consider those with at least some education
data_2019 = data_2019[data_2019["EdLevel"] != "I never completed any formal education"]

data_2019 = data_2019.fillna("no_answer")

num_columns = ["Age", "Age1stCode", "YearsCode", "WorkWeekHrs", "CodeRevHrs"]

# Convert numeric columns to int
for col in num_columns:
    data_2019[col] = data_2019[col].astype("int32", errors="ignore")

data_2019["Age1stCode"].replace("Younger than 5 years", "4", inplace=True)
data_2019["YearsCode"].replace("Less than 1 year", "0", inplace=True)
data_2019["YearsCode"].replace("More than 50 years", "51", inplace=True)

for col in num_columns:
    data_2019[col] = data_2019[col].apply(string_replace)

# Exclude respondents who selected multiple gender, race, or sexual orientation
# options
data_2019 = data_2019[~data_2019["Gender"].str.contains(";")]
data_2019 = data_2019[~data_2019["Ethnicity"].str.contains(";")]
data_2019 = data_2019[~data_2019["Sexuality"].str.contains(";")]

# Reset index
data_2019 = data_2019.reset_index(drop=True)

In [554]:
# TODO: Bucketing for numeric responses (Age, Age1stCode, YearsCode, WorkWeekHrs, CodeRevHrs)

age_labels = ["no_answer","-20", "21-25", "26-30", "31-35", "35-40", "41-45", "45-50", "51-55", "55-60", "61-65", "66-"]
year_labels = ["no_answer","0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "35-40", "41-"]
work_week_hour_labels = ["no_answer","-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", "81-"]
code_rev_hour_labels = ["no_answer","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11-15", "16-20", "21-"]

age_buckets = np.array([-1001,0,20,25,30,35,40,45,50,55,60,65,100])
year_buckets = np.array([-1001,0,5,10,15,20,25,30,35,40,100])
work_week_hour_buckets = np.array([-1001,0,10,20,30,40,50,60,70,80,200])
code_rev_hour_buckets = np.array([-1001,0,1,2,3,4,5,6,7,8,9,10,15,20,200])

In [555]:
data_2019["Age"] = pd.cut(data_2019["Age"], age_buckets, labels=age_labels).astype("str")
data_2019["Age1stCode"] = pd.cut(data_2019["Age1stCode"], age_buckets, labels=age_labels).astype("str")
data_2019["YearsCode"] = pd.cut(data_2019["YearsCode"], year_buckets, labels=year_labels).astype("str")
data_2019["WorkWeekHrs"] = pd.cut(data_2019["WorkWeekHrs"], work_week_hour_buckets, labels=work_week_hour_labels).astype("str")
data_2019["CodeRevHrs"] = pd.cut(data_2019["CodeRevHrs"], code_rev_hour_buckets, labels=code_rev_hour_labels).astype("str")

In [560]:
controls = create_controls(data_2019, "Income")

In [564]:
data_2019 = design_matrix(data_2019, controls)

In [561]:
controls

{'Age': {'controls': ['35_40',
   '45_50',
   '51_55',
   '61_65',
   '55_60',
   '31_35',
   '66_',
   '21_25',
   '41_45',
   '26_30',
   '_20'],
  'omitted': '26_30'},
 'Age1stCode': {'controls': ['no_answer',
   '45_50',
   '41_45',
   '51_55',
   '31_35',
   '21_25',
   '35_40',
   '26_30',
   '_20'],
  'omitted': '_20'},
 'CodeRev': {'controls': ['yes_because_i_see_value_in_code_review',
   'no',
   'yes_because_i_was_told_to_do_so',
   'no_answer'],
  'omitted': 'yes_because_i_see_value_in_code_review'},
 'CodeRevHrs': {'controls': ['16_20',
   'no_answer',
   '5',
   '11_15',
   '9',
   '6',
   '21_',
   '2',
   '7',
   '4',
   '1',
   '8',
   '3',
   '10'],
  'omitted': 'no_answer'},
 'Containers': {'controls': ['no_answer',
   'testing',
   'i_do_not_use_containers',
   'development',
   'outside_of_work_for_personal_projects',
   'production'],
  'omitted': 'i_do_not_use_containers'},
 'Country': {'controls': ['united_states'], 'omitted': 'united_states'},
 'DatabaseDesireNe